In [1]:
import pandas as pd
import numpy as np
import time
import json
import requests
from datetime import date
from matplotlib import pyplot as plt
import datetime
import timeit

In [2]:
# ADD API KEYS HERE - they will all be used in API calls as to avoid rate limiting
api_keys = ['31bf3a0ba5585b5f5de37aedef10ffcd',
            '855d1f6526f52f62fc4aeec74354a62e',
            '7de8bd3327159f7385e23508b0b71419',
            '5a767fa6f202e0f08d366232f99e8d3c',
            'ef64740368755f761543d05127051589',
            'af06c12c93dd48337853ff2daf61e8ef']

In [3]:
state_level_feat = pd.read_csv('state_features_final.csv')

In [4]:
chosen_state_feat = np.unique(state_level_feat.title.values)
print(np.unique(state_level_feat['frequency']))
print(len(chosen_state_feat))
# print(chosen_state_feat)

['Annual' 'Monthly' 'Quarterly' 'Quarterly, End of Period'
 'Weekly, Ending Saturday']
181


In [14]:
state_level_feat.head()
# state_list = np.unique(state_level_feat['state_id'].values)
# print(state_list)
# to_retry = np.unique([27300])

,Unnamed: 0,id,title,observation_start,observation_end,frequency,units,seasonal_adjustment,state_id
0,3,AGEXMAL1A647NCEN,Age 65 and Over Tax Exemptions for,1989-01-01,2017-01-01,Annual,Number of Exemptions,Not Seasonally Adjusted,27282
1,45,ALALLL,Allowance for Loan and Lease Losses for Commer...,1984-01-01,2019-10-01,Quarterly,Thousands of Dollars,Not Seasonally Adjusted,27282
2,77,ALATRR,Allocated Transfer Risk Reserves for Commercia...,1984-01-01,2019-10-01,Quarterly,Thousands of Dollars,Not Seasonally Adjusted,27282
3,110,ALBP1FHSA,New Private Housing Units Authorized by Buildi...,1988-01-01,2020-02-01,Monthly,Units,Seasonally Adjusted,27282
4,112,ALBPPRIVSA,New Private Housing Units Authorized by Buildi...,1988-01-01,2020-02-01,Monthly,Units,Seasonally Adjusted,27282


In [7]:
# edit the start and end variables to determine which counties to get data for
start = 0
end = 1
count = start

state_list = np.unique(state_level_feat['state_id'].values)
feat_ids = state_level_feat['id'].values
cur_key_num = 0
num_keys = len(api_keys)
failed_ids = []
for state_id in to_retry:
    start_time = timeit.default_timer()
#     try:
    time.sleep(.1)
    params = {
        'category_id': state_id,
        'api_key': api_keys[cur_key_num],
        'file_type': 'json'
    }
    cur_key_num = (cur_key_num + 1) % num_keys

    r = requests.get(url = 'https://api.stlouisfed.org/fred/category/series', params=params)
    res_1000 = r.json()
    print(len(res_1000['seriess']))

    time.sleep(.1)
    params = {
        'category_id': state_id,
        'api_key': api_keys[cur_key_num],
        'file_type': 'json',
        'offset': 1000
    }
    cur_key_num = (cur_key_num + 1) % num_keys

    r2 = requests.get(url = 'https://api.stlouisfed.org/fred/category/series', params=params)
    res_2000 = r2.json()
    print(len(res_2000['seriess']))

    # Get the observations for a series
    seriess_1000 = res_1000['seriess']
    seriess_2000 = res_2000['seriess']
    seriess = seriess_1000 + seriess_2000

    res2 = []
    for series in range(len(seriess)):
        current_series = seriess[series]
        series_id = current_series['id']
        freq = current_series['frequency']

        if (freq != "Weekly, Ending Saturday"):
            params = {
                'series_id': series_id,
                'api_key': api_keys[cur_key_num],
                'file_type': 'json'
            }
        else:
            params = {
                'series_id': series_id,
                'api_key': api_keys[cur_key_num],
                'file_type': 'json',
                'frequency': 'm',
                'aggregation_method': 'avg'
            }
        cur_key_num = (cur_key_num + 1) % num_keys
        time.sleep(.1)

        if series_id in feat_ids:
            r = requests.get(url = 'https://api.stlouisfed.org/fred/series/observations', params=params)

            res1 = r.json()
            res1["observations"] = [i for i in res1["observations"] if not (int(i["date"][0:4]) < 2010 or int(i["date"][0:4]) > 2017)]

            for val in res1["observations"]:
                del val["realtime_start"]
                del val["realtime_end"]


            for val in res1['observations']:
                if (freq == "Annual"):
                    val['feature'] = series_id
                    res2 += [val]
                    for i in range(2, 13):
                        if i < 10:
                            string = '0' + str(i)
                        else:
                            string = str(i)
                        dict_to_add = {}
                        dict_to_add['feature'] = series_id
                        dict_to_add['date'] = val['date'][0:5] + string + val['date'][7:10]
                        dict_to_add['value'] = val['value']
                        res2 += [dict_to_add]
                elif (freq == "Quarterly" or freq == "Quarterly, End of Period"):
                    val['feature'] = series_id
                    res2 += [val]
                    for i in range(1, 3):
                        cur_date = int(val['date'][5:7])
                        date = i + cur_date
                        if date < 10:
                            date = '0' + str(date)
                        else:
                            date = str(date)

                        dict_to_add = {}
                        dict_to_add['feature'] = series_id
                        dict_to_add['date'] = val['date'][0:5] + date + val['date'][7:10]
                        dict_to_add['value'] = val['value']
                        res2 += [dict_to_add]
                elif (freq == "Weekly, Ending Saturday"):
                    val['feature'] = series_id
                    res2 += [val]
                else:
                    val['feature'] = series_id
                    res2 += [val]
    dates = []
    for i in range(96):
        dates += [res2[i]['date']]

    res3 = {}
    for val in res2:
        if val['feature'] not in res3.keys():
            # new thing
            res3[val['feature']] = [val['value']]
        else:
            res3[val['feature']] += [val['value']]

    for feat in res3:
        if len(res3[feat]) != 96:
            del res3[feat]

    name = str(state_id) + '.csv'

    data = pd.DataFrame(res3, index=dates)
    data.to_csv(name)
    elapsed = timeit.default_timer() - start_time
    print('%d. csv written, state id: %s \t time elapsed: %f' %(count, state_id, elapsed))
    count += 1

#     except:
#         print('State ID Failed, rate limited:', state_id)
#         failed_ids += [state_id]

print('Failed counties:')
if (len(failed_ids)) == 0:
    print("None")
else:
    for i in failed_ids:
        print(i)

1000
665


RuntimeError: dictionary changed size during iteration

In [21]:
state_level_feat.head()
title_dict = {}
for index, row in state_level_feat.iterrows():
    title_dict[row["id"]] = row['title']
# for item in title_dict:
#     print(item, title_dict[item])


In [63]:
# code to rename the columns in the csvs
import os
csvs = os.listdir('state_data')

start_time = timeit.default_timer()

# print(csvs)
for idx, file in enumerate(csvs):
    df = pd.read_csv(('state_data/' + file), index_col=0)
    df.rename(columns=title_dict, inplace=True)
    df = df.reindex(sorted(df.columns), axis=1)
    df.to_csv(('state_data/' + file))
    if (idx + 1) % 100 == 0:
        print(idx+1)
elapsed = timeit.default_timer() - start_time

['149.csv', '150.csv', '151.csv', '152.csv', '153.csv', '154.csv', '193.csv', '27282.csv', '27284.csv', '27286.csv', '27287.csv', '27289.csv', '27291.csv', '27292.csv', '27293.csv', '27294.csv', '27297.csv', '27298.csv', '27301.csv', '27302.csv', '27304.csv', '27305.csv', '27308.csv', '27309.csv', '27310.csv', '27311.csv', '27312.csv', '27313.csv', '27314.csv', '27315.csv', '27316.csv', '27317.csv', '27318.csv', '27319.csv', '27320.csv', '27323.csv', '27324.csv', '27326.csv', '27328.csv', '27329.csv', '27330.csv', '27331.csv', '27332.csv', '27333.csv', '27334.csv']
          Accommodation and Food Services Earnings in  \
1/1/2010                                      1587780   
2/1/2010                                      1587780   
3/1/2010                                      1587780   
4/1/2010                                      1619540   
5/1/2010                                      1619540   

          Accommodation and Food Services Wages and Salaries in  \
1/1/2010          

            Accommodation and Food Services Earnings in  \
2010-01-01                                    4011468.0   
2010-02-01                                    4011468.0   
2010-03-01                                    4011468.0   
2010-04-01                                    4043136.0   
2010-05-01                                    4043136.0   

            Accommodation and Food Services Wages and Salaries in  \
2010-01-01                                          3451848.0       
2010-02-01                                          3451848.0       
2010-03-01                                          3451848.0       
2010-04-01                                          3482356.0       
2010-05-01                                          3482356.0       

            Administrative and Waste Services Earnings in  \
2010-01-01                                      5741800.0   
2010-02-01                                      5741800.0   
2010-03-01                                     

            Accommodation and Food Services Earnings in  \
2010-01-01                                    2334044.0   
2010-02-01                                    2334044.0   
2010-03-01                                    2334044.0   
2010-04-01                                    2364640.0   
2010-05-01                                    2364640.0   

            Accommodation and Food Services Wages and Salaries in  \
2010-01-01                                          1909816.0       
2010-02-01                                          1909816.0       
2010-03-01                                          1909816.0       
2010-04-01                                          1934280.0       
2010-05-01                                          1934280.0       

            Administrative and Waste Services Earnings in  \
2010-01-01                                      1764140.0   
2010-02-01                                      1764140.0   
2010-03-01                                     

            Accommodation and Food Services Earnings in  \
2010-01-01                                    4923720.0   
2010-02-01                                    4923720.0   
2010-03-01                                    4923720.0   
2010-04-01                                    4987596.0   
2010-05-01                                    4987596.0   

            Accommodation and Food Services Wages and Salaries in  \
2010-01-01                                          4063720.0       
2010-02-01                                          4063720.0       
2010-03-01                                          4063720.0       
2010-04-01                                          4120320.0       
2010-05-01                                          4120320.0       

            Administrative and Waste Services Earnings in  \
2010-01-01                                      7808852.0   
2010-02-01                                      7808852.0   
2010-03-01                                     

            Accommodation and Food Services Earnings in  \
2010-01-01                                    5341596.0   
2010-02-01                                    5341596.0   
2010-03-01                                    5341596.0   
2010-04-01                                    5456688.0   
2010-05-01                                    5456688.0   

            Accommodation and Food Services Wages and Salaries in  \
2010-01-01                                          4334772.0       
2010-02-01                                          4334772.0       
2010-03-01                                          4334772.0       
2010-04-01                                          4421548.0       
2010-05-01                                          4421548.0       

            Administrative and Waste Services Earnings in  \
2010-01-01                                      8670604.0   
2010-02-01                                      8670604.0   
2010-03-01                                     

...

            Accommodation and Food Services Earnings in  \
2010-01-01                                    3602812.0   
2010-02-01                                    3602812.0   
2010-03-01                                    3602812.0   
2010-04-01                                    3614860.0   
2010-05-01                                    3614860.0   

            Accommodation and Food Services Wages and Salaries in  \
2010-01-01                                          3040024.0       
2010-02-01                                          3040024.0       
2010-03-01                                          3040024.0       
2010-04-01                                          3057044.0       
2010-05-01                                          3057044.0       

            Administrative and Waste Services Earnings in  \
2010-01-01                                      5211204.0   
2010-02-01                                      5211204.0   
2010-03-01                                     

            Age 65 and Over Tax Exemptions for  \
2010-01-01                             1644704   
2010-02-01                             1644704   
2010-03-01                             1644704   
2010-04-01                             1644704   
2010-05-01                             1644704   

            Age 65 and Over, Poverty Tax Exemptions for  \
2010-01-01                                       300622   
2010-02-01                                       300622   
2010-03-01                                       300622   
2010-04-01                                       300622   
2010-05-01                                       300622   

            All Employees: Transportation and Utilities: Transportation, Warehousing, and Utilities in  \
2010-01-01                                              413.8                                            
2010-02-01                                              414.8                                            
2010-03-01                

            Accommodation and Food Services Earnings in  \
2010-01-01                                     653140.0   
2010-02-01                                     653140.0   
2010-03-01                                     653140.0   
2010-04-01                                     671200.0   
2010-05-01                                     671200.0   

            Accommodation and Food Services Wages and Salaries in  \
2010-01-01                                           549372.0       
2010-02-01                                           549372.0       
2010-03-01                                           549372.0       
2010-04-01                                           565120.0       
2010-05-01                                           565120.0       

            Administrative and Waste Services Earnings in  \
2010-01-01                                       420924.0   
2010-02-01                                       420924.0   
2010-03-01                                     

            Accommodation and Food Services Earnings in  \
2010-01-01                                    5163224.0   
2010-02-01                                    5163224.0   
2010-03-01                                    5163224.0   
2010-04-01                                    5298960.0   
2010-05-01                                    5298960.0   

            Accommodation and Food Services Wages and Salaries in  \
2010-01-01                                          4346896.0       
2010-02-01                                          4346896.0       
2010-03-01                                          4346896.0       
2010-04-01                                          4459448.0       
2010-05-01                                          4459448.0       

            Administrative and Waste Services Earnings in  \
2010-01-01                                      7597076.0   
2010-02-01                                      7597076.0   
2010-03-01                                     

            Accommodation and Food Services Earnings in  \
2010-01-01                                    3585872.0   
2010-02-01                                    3585872.0   
2010-03-01                                    3585872.0   
2010-04-01                                    3603256.0   
2010-05-01                                    3603256.0   

            Accommodation and Food Services Wages and Salaries in  \
2010-01-01                                          2973732.0       
2010-02-01                                          2973732.0       
2010-03-01                                          2973732.0       
2010-04-01                                          2998352.0       
2010-05-01                                          2998352.0       

            Administrative and Waste Services Earnings in  \
2010-01-01                                      4584208.0   
2010-02-01                                      4584208.0   
2010-03-01                                     

In [64]:
# IGNORE: messing with code, other ways to do it

# Get datasets

# Set up DataFrame
country_features = pd.DataFrame(columns=['date'])
curr_state = ""
start = 0
end = 181

for series_id, series_name, freq, state in zip(country_series_list[start:end], country_series_titles[start:end], country_series_freq[start:end], country_series_state[start:end]):
#     print(state)
    time.sleep(0.5)
    if (freq != "Weekly, Ending Saturday"):
        params = {
            'series_id': series_id,
            'api_key': 'd4bf33a07481ae1c55ca182cf5d157fb',
            'file_type': 'json'
        }
    else:
        params = {
            'series_id': series_id,
            'api_key': 'd4bf33a07481ae1c55ca182cf5d157fb',
            'file_type': 'json',
            'frequency': 'm',
            'aggregation_method': 'avg'
        }
    r = requests.get(url = 'https://api.stlouisfed.org/fred/series/observations', params=params)
    
    res = r.json()
    
    if (freq == "Annual"):
        val['feature'] = series_id
        res2 += [val]
        for i in range(2, 13):
            if i < 10:
                string = '0' + str(i)
            else:
                string = str(i)
            dict_to_add = {}
            dict_to_add['feature'] = series_id
            dict_to_add['date'] = val['date'][0:5] + string + val['date'][7:10]
            dict_to_add['value'] = val['value']
            res2 += [dict_to_add]
    elif (freq == "Quarterly" or freq == "Quarterly, End of Period"):
        val['feature'] = series_id
        res2 += [val]
        for i in range(1, 3):
            cur_date = int(val['date'][5:7])
            date = i + cur_date
            if date < 10:
                date = '0' + str(date)
            else:
                date = str(date)

            dict_to_add = {}
            dict_to_add['feature'] = series_id
            dict_to_add['date'] = val['date'][0:5] + date + val['date'][7:10]
            dict_to_add['value'] = val['value']
            res2 += [dict_to_add]
    elif (freq == "Weekly, Ending Saturday"):
        val['feature'] = series_id
        res2 += [val]
    else:
        val['feature'] = series_id
        res2 += [val] 

    df_current = pd.DataFrame(res['observations'])
    df_current.rename(columns={'value': series_name}, inplace=True)
    df_current.drop(['realtime_start', 'realtime_end'], axis=1, inplace=True)
    country_features = pd.merge(country_features, df_current, how='outer', on=['date'])
    
country_features.to_csv('state_features_test.csv', index=False)

KeyboardInterrupt: 